# COVID-19 and Weather Patterns

## Imports

- *os* for interfacing with the operating system
- *pathlib* for interfacing with the file system
- *zipfile* for managing archive files

- *numpy* for array processing
- *pandas* for tabular processing
- *tensorflow* for tensor processing
- *keras* for simplified tensor processing

- *matplotlib* for visualization
- *seaborn* for enhanced visualization

In [1]:
# Custom
import data_processing

# File System
import os
import json
from pathlib import Path
from zipfile import ZipFile

# Processing
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

tensorflow.__version__

'2.7.0'

## Introduction

The goal of this work is to determine whether or not weather patterns should be considered as a meaningful supporting input when making predictions about new daily COVID-19 cases within a given geographical space. Using census, weather, and COVID-19 datasets provided by the Urban Sustain project, the authors attempt to quantify the correlation between particular weather patterns and COVID-19 transmission events.

## Defining Terms

***Urban Sustain Project*** - A joint effort between researchers at Colorado State University, Arizona State University, the University of California-Irvine, and the University of Maryland – Baltimore County.

## Loading Data

**DEVELOPER NOTE:** Download the five required datasets from Urban Sustain and place them in the cloned repository at ```./data/```. These datasets are also available at a shared OneDrive folder. This logic expects that these files exist at relative path ```../data/``` with respect to this notebook.

We'll begin by defining a path to our data directory and a list of the datasets that we expect to find there.

In [2]:
DATA_PATH = '../data/' # Point this constant to the location of your data archive files

EXPECTED_DATASETS = [
    'county_total_population.Colorado.zip',
    'covid_county.Colorado.zip',
    'neon_2d_wind.Colorado.zip',
    'neon_barometric_pressure.Colorado.zip',
    'neon_single_asp_air_temperature.Colorado.zip'
]

Next, we will attempt to extract each of these archived datasets into a dedicated subdirectory within the given data directory.

In [3]:
# For each listed dataset string in the EXPECTED_DATASETS constant
for datasetName in EXPECTED_DATASETS:
    try:
        # Open the given archive file
        with ZipFile(DATA_PATH + datasetName, 'r') as currentZip:
            # Build the target directory path for extracted data
            datasetNameTokens = datasetName.split('.')
            datasetNameTokens.remove('zip')
            targetDirectory = DATA_PATH + '.'.join(datasetNameTokens)
            
            # If the target directory doesn't exist, create it
            if not os.path.exists(targetDirectory):
                Path(targetDirectory).mkdir()
            
            # Extract all data from the archive file to the target directory
            currentZip.extractall(targetDirectory)
    except FileNotFoundError:
        print("Unable to open " + datasetName + " at path " + DATA_PATH + datasetName)

## File Information

Four of the five datasets referenced in this work relate geospatial information to particular events as they occur over time Each is provided by the Urban Sustain project and employs a similar file structure.

- data.json
- fieldLabels.json
- linkedGeometry.json
- README.txt


#### New York Times COVID-19 County Dataset

The included COVID-19 County dataset 

In [4]:
covidDataFrame = pd.io.json.read_json('../data/covid_county.Colorado/data.json')
covidDataFrame.sample(5)

,_id,GISJOIN,epoch_time,dateString,county,state,totalCaseCount,newCaseCount,totalDeathCount,newDeathCount
6416,{'$oid': '615fc624bf06594c4cad5b34'},G0800190,{'$numberLong': '1589004000000'},5/9/20,Clear Creek,Colorado,14,0,1,0
13531,{'$oid': '615fc625bf06594c4cad7484'},G0800410,{'$numberLong': '1610002800000'},1/7/21,El Paso,Colorado,43156,350,614,8
31802,{'$oid': '615fc627bf06594c4cadc0e1'},G0800990,{'$numberLong': '1627797600000'},8/1/21,Prowers,Colorado,1296,0,22,0
35427,{'$oid': '615fc628bf06594c4cadceaf'},G0801110,{'$numberLong': '1609657200000'},1/3/21,San Juan,Colorado,27,0,0,0
25388,{'$oid': '615fc626bf06594c4cada5ce'},G0800790,{'$numberLong': '1622008800000'},5/26/21,Mineral,Colorado,93,0,1,0


In [5]:
covidLabelsFrame = pd.io.json.read_json('../data/covid_county.Colorado/fieldLabels.json')
covidLabelsFrame.sample(5)

,name,label
3,newCaseCount,New Case Count
1,newDeathCount,New Death Count
4,totalCaseCount,Total Case Count
2,totalDeathCount,Total Death Count
0,epoch_time,Date


In [6]:
covidGeometryFrame = pd.io.json.read_json('../data/covid_county.Colorado/linkedGeometry.json')
covidGeometryFrame.sample(5)

,_id,type,geometry,properties,GISJOIN
48,{'$oid': '604964eefed2e95d6b68b833'},Feature,"{'type': 'Polygon', 'coordinates': [[[-102.051...","{'STATEFP10': '08', 'COUNTYFP10': '095', 'COUN...",G0800950
51,{'$oid': '604964eefed2e95d6b68b838'},Feature,"{'type': 'Polygon', 'coordinates': [[[-104.053...","{'STATEFP10': '08', 'COUNTYFP10': '101', 'COUN...",G0801010
6,{'$oid': '604964eefed2e95d6b68b844'},Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...","{'STATEFP10': '08', 'COUNTYFP10': '013', 'COUN...",G0800130
60,{'$oid': '604964eefed2e95d6b68b840'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.329...","{'STATEFP10': '08', 'COUNTYFP10': '119', 'COUN...",G0801190
18,{'$oid': '604964eefed2e95d6b68b839'},Feature,"{'type': 'Polygon', 'coordinates': [[[-104.660...","{'STATEFP10': '08', 'COUNTYFP10': '035', 'COUN...",G0800350


#### NEON 2D Wind Dataset

In [7]:
windDataFrame = pd.io.json.read_json('../data/neon_2d_wind.Colorado/data.json')
windDataFrame.sample(5)

,_id,site,epoch_time,startDateTime,endDateTime,windSpeedMean,windSpeedMinimum,windSpeedMaximum,windSpeedVariance,windSpeedNumPts,windSpeedExpUncert,windSpeedStdErMean,windSpeedFinalQF,windDirMean,windDirVariance,windDirNumPts,windDirExpUncert,windDirStdErMean,windDirFinalQF
448688,{'$oid': '60cbc07651c1683b462a1272'},CPER_DP1.00001.001_000.030,{'$numberLong': '1494592200000'},2017-05-12T12:30:00Z,2017-05-12T13:00:00Z,0.60,0.03,1.30,0.06,1800,0.03,0.01,0,69.97,1066.08,1800,0.07,0.05,1
892945,{'$oid': '60cbc2a551c1683b4666d9a3'},RMNP_DP1.00001.001_000.020,{'$numberLong': '1614448800000'},2021-02-27T18:00:00Z,2021-02-27T18:30:00Z,0.76,0.01,2.99,0.17,1800,0.06,0.01,0,81.64,2744.46,1800,0.07,0.05,1
1204834,{'$oid': '60cbc05551c1683b462685b1'},STER_DP1.00001.001_000.010,{'$numberLong': '1573932600000'},2019-11-16T19:30:00Z,2019-11-16T20:00:00Z,4.52,1.87,8.52,1.08,1800,0.29,0.02,0,337.71,173.28,1800,0.19,0.03,0
175751,{'$oid': '60cbc06851c1683b46288cb5'},CPER_DP1.00001.001_000.010,{'$numberLong': '1430807400000'},2015-05-05T06:30:00Z,2015-05-05T07:00:00Z,1.32,0.40,2.56,0.12,1796,0.05,0.01,0,115.83,134.19,1796,0.07,0.00,0
920601,{'$oid': '60cbc27c51c1683b466248ff'},RMNP_DP1.00001.001_000.030,{'$numberLong': '1465882200000'},2016-06-14T05:30:00Z,2016-06-14T06:00:00Z,1.90,0.01,7.59,1.44,1800,0.24,0.03,0,31.68,747.20,1800,0.07,0.06,1


In [8]:
windLabelsFrame = pd.io.json.read_json('../data/neon_2d_wind.Colorado/fieldLabels.json')
windLabelsFrame.sample(5)

,name,label
17,windSpeedNullPassQM,Wind speed quality metric that summarizes the ...
58,windDirCalmWindQF,Wind direction quality flag for the calm wind ...
23,windSpeedSensorErrorPassQM,Wind speed quality metric that summarizes the ...
51,windDirDistortedFlowPassQM,Wind direction quality metric that summarizes ...
57,windDirBetaQM,Wind direction quality metric detailing the ou...


In [9]:
windGeometryFrame = pd.io.json.read_json('../data/neon_2d_wind.Colorado/linkedGeometry.json')
windGeometryFrame.sample(5)

,_id,type,geometry,properties,site
13,{'$oid': '60cbcf5eed5577c25c205208'},Feature,"{'type': 'Point', 'coordinates': [-103.02929, ...","{'HOR.VER': '000.010', 'name': 'CFGLOC100113',...",STER_DP1.00001.001_000.010
1,{'$oid': '60cbcf5eed5577c25c205233'},Feature,"{'type': 'Point', 'coordinates': [-105.545955,...","{'HOR.VER': '000.040', 'name': 'CFGLOC100675',...",RMNP_DP1.00001.001_000.040
4,{'$oid': '60cbcf5eed5577c25c205282'},Feature,"{'type': 'Point', 'coordinates': [-105.544602,...","{'HOR.VER': '200.000', 'name': 'CFGLOC101555',...",COMO_DP1.00001.001_200.000
9,{'$oid': '60cbcf5eed5577c25c20520d'},Feature,"{'type': 'Point', 'coordinates': [-104.74559, ...","{'HOR.VER': '000.030', 'name': 'CFGLOC100245',...",CPER_DP1.00001.001_000.030
5,{'$oid': '60cbcf5eed5577c25c205294'},Feature,"{'type': 'Point', 'coordinates': [-105.58238, ...","{'HOR.VER': '000.010', 'name': 'CFGLOC109345',...",NIWO_DP1.00001.001_000.010


#### NEON Barometric Pressure Dataset

In [10]:
pressureDataFrame = pd.io.json.read_json('../data/neon_barometric_pressure.Colorado/data.json')
pressureDataFrame.sample(5)

,_id,site,epoch_time,startDateTime,endDateTime,staPresMean,staPresMinimum,staPresMaximum,staPresVariance,staPresNumPts,staPresExpUncert,staPresStdErMean,staPresFinalQF,corPres,corPresExpUncert,corPresFinalQF
300016,{'$oid': '60c78c98c8cb4f3f3b1a57bd'},RMNP_DP1.00004.001,{'$numberLong': '1456225200000'},2016-02-23T11:00:00Z,2016-02-23T11:30:00Z,72.77423,72.75630,72.78930,0.00009,180,0.01726,0.00072,0,102.66271,0.46474,0
420995,{'$oid': '60c78cddc8cb4f3f3b225ef1'},STER_DP1.00004.001,{'$numberLong': '1498084200000'},2017-06-21T22:30:00Z,2017-06-21T23:00:00Z,85.78679,85.76873,85.79874,0.00009,180,0.01725,0.00069,0,99.48795,0.08785,0
514155,{'$oid': '60c78d4ac8cb4f3f3b2eb276'},WLOU_DP1.00004.001,{'$numberLong': '1571529600000'},2019-10-20T00:00:00Z,2019-10-20T00:30:00Z,70.69050,70.67899,70.69900,0.00003,180,0.04729,0.00037,0,100.50499,0.48701,0
245681,{'$oid': '60c78c29c8cb4f3f3b0d99d9'},NIWO_DP1.00004.001,{'$numberLong': '1519279200000'},2018-02-22T06:00:00Z,2018-02-22T06:30:00Z,65.65192,65.59410,65.67409,0.00057,180,0.01756,0.00177,0,101.55398,0.73793,0
370631,{'$oid': '60c78ca1c8cb4f3f3b1b6c38'},RMNP_DP1.00004.001,{'$numberLong': '1590022800000'},2020-05-21T01:00:00Z,2020-05-21T01:30:00Z,72.50854,72.48516,72.52516,0.00008,180,0.04723,0.00066,0,99.73133,0.39901,0


In [11]:
pressureLabelsFrame = pd.io.json.read_json('../data/neon_barometric_pressure.Colorado/fieldLabels.json')
pressureLabelsFrame.sample(5)

,name,label
14,staPresPersistencePassQM,Station pressure quality metric that summarize...
30,staPresValidCalNAQM,Station pressure quality metric that summarize...
13,staPresPersistenceFailQM,Station pressure quality metric that summarize...
24,staPresGapNAQM,Station pressure quality metric that summarize...
36,corPresExpUncert,Expanded uncertainty for pressure corrected to...


In [12]:
pressureGeometryFrame = pd.io.json.read_json('../data/neon_barometric_pressure.Colorado/linkedGeometry.json')
pressureGeometryFrame.sample(5)

,_id,type,geometry,properties,site
5,{'$oid': '60c78a9d230c7f49850c20e3'},Feature,"{'type': 'Point', 'coordinates': [-103.02929, ...","{'site': 'STER_DP1.00004.001', 'name': 'neon_b...",STER_DP1.00004.001
3,{'$oid': '60c78a9d230c7f49850c20cf'},Feature,"{'type': 'Point', 'coordinates': [-105.58238, ...","{'site': 'NIWO_DP1.00004.001', 'name': 'neon_b...",NIWO_DP1.00004.001
4,{'$oid': '60c78a9d230c7f49850c20b1'},Feature,"{'type': 'Point', 'coordinates': [-104.74559, ...","{'site': 'CPER_DP1.00004.001', 'name': 'neon_b...",CPER_DP1.00004.001
2,{'$oid': '60c78a9d230c7f49850c20b0'},Feature,"{'type': 'Point', 'coordinates': [-105.544602,...","{'site': 'COMO_DP1.00004.001', 'name': 'neon_b...",COMO_DP1.00004.001
1,{'$oid': '60c78a9d230c7f49850c20ef'},Feature,"{'type': 'Point', 'coordinates': [-105.914674,...","{'site': 'WLOU_DP1.00004.001', 'name': 'neon_b...",WLOU_DP1.00004.001


#### NEON Air Temperature

In [13]:
temperatureDataFrame = pd.io.json.read_json('../data/neon_single_asp_air_temperature.Colorado/data.json')
temperatureDataFrame.sample(5)

,_id,site,epoch_time,startDateTime,endDateTime,tempSingleMean,tempSingleMinimum,tempSingleMaximum,tempSingleVariance,tempSingleNumPts,tempSingleExpUncert,tempSingleStdErMean,finalQF
408145,{'$oid': '60cbd3d50fc3818c81e0477c'},CPER_DP1.00002.001_000.030,{'$numberLong': '1568102400000'},2019-09-10T08:00:00Z,2019-09-10T08:30:00Z,15.5113,14.3815,16.1919,0.2241,1800,0.1428,0.0112,0
452157,{'$oid': '60cbd8960fc3818c81705829'},NIWO_DP1.00002.001_000.010,{'$numberLong': '1545654600000'},2018-12-24T12:30:00Z,2018-12-24T13:00:00Z,-9.8435,-9.9490,-9.7095,0.0028,1800,0.1300,0.0013,0
67017,{'$oid': '60cbd7430fc3818c814867e5'},ARIK_DP1.00002.001_200.000,{'$numberLong': '1586151000000'},2020-04-06T05:30:00Z,2020-04-06T06:00:00Z,8.4192,7.7203,9.2041,0.1648,1800,0.1393,0.0096,0
992734,{'$oid': '60cbd3a20fc3818c81da4caa'},STER_DP1.00002.001_000.010,{'$numberLong': '1496518200000'},2017-06-03T19:30:00Z,2017-06-03T20:00:00Z,23.3519,22.1351,24.4304,0.3940,1800,0.1477,0.0148,0
154067,{'$oid': '60cbd81c0fc3818c81620a54'},COMO_DP1.00002.001_200.000,{'$numberLong': '1619713800000'},2021-04-29T16:30:00Z,2021-04-29T17:00:00Z,5.4967,4.9427,6.4060,0.0821,509,0.1418,0.0127,1


In [14]:
temperatureLabelsFrame = pd.io.json.read_json('../data/neon_single_asp_air_temperature.Colorado/fieldLabels.json')
temperatureLabelsFrame.sample(5)

,name,label
40,finalQFSciRvw,Quality flag indicating whether a data product...
39,finalQF,Quality flag indicating whether a data product...
20,nullPassQM,Quality metric that summarizes the passed outc...
11,rangePassQM,Quality metric that summarizes the passed outc...
22,gapFailQM,Quality metric that summarizes the failed outc...


In [15]:
temperatureGeometryFrame = pd.io.json.read_json('../data/neon_single_asp_air_temperature.Colorado/linkedGeometry.json')
temperatureGeometryFrame.sample(5)

,_id,type,geometry,properties,site
6,{'$oid': '60cbd3907adbc5ff35cc422f'},Feature,"{'type': 'Point', 'coordinates': [-105.58238, ...","{'HOR.VER': '000.030', 'name': 'CFGLOC109358',...",NIWO_DP1.00002.001_000.030
3,{'$oid': '60cbd3907adbc5ff35cc41c9'},Feature,"{'type': 'Point', 'coordinates': [-105.545955,...","{'HOR.VER': '000.010', 'name': 'CFGLOC100665',...",RMNP_DP1.00002.001_000.010
12,{'$oid': '60cbd3907adbc5ff35cc41a1'},Feature,"{'type': 'Point', 'coordinates': [-103.02929, ...","{'HOR.VER': '000.010', 'name': 'CFGLOC100016',...",STER_DP1.00002.001_000.010
0,{'$oid': '60cbd3907adbc5ff35cc41cb'},Feature,"{'type': 'Point', 'coordinates': [-105.545955,...","{'HOR.VER': '000.030', 'name': 'CFGLOC100671',...",RMNP_DP1.00002.001_000.030
9,{'$oid': '60cbd3907adbc5ff35cc41a6'},Feature,"{'type': 'Point', 'coordinates': [-104.74559, ...","{'HOR.VER': '000.030', 'name': 'CFGLOC100244',...",CPER_DP1.00002.001_000.030


### U.S. Census Total County Population Dataset

In [16]:
populationDataFrame = pd.io.json.read_json('../data/county_total_population.Colorado/data.json')
populationDataFrame

,_id,GISJOIN,STATE,STATEFP,STATENH,COUNTY,COUNTYFP,COUNTYNH,1980_total_population,1990_total_population,2000_total_population,2010_total_population
0,{'$oid': '607d4c2708e81eceb6b6e384'},G0800010,Colorado,8,80,Adams County,1,10,245944,265038,363857,441603
1,{'$oid': '607d4c2708e81eceb6b6e385'},G0800030,Colorado,8,80,Alamosa County,3,30,11799,13617,14966,15445
2,{'$oid': '607d4c2708e81eceb6b6e386'},G0800050,Colorado,8,80,Arapahoe County,5,50,293621,391511,487967,572003
3,{'$oid': '607d4c2708e81eceb6b6e387'},G0800070,Colorado,8,80,Archuleta County,7,70,3664,5345,9898,12084
4,{'$oid': '607d4c2708e81eceb6b6e388'},G0800090,Colorado,8,80,Baca County,9,90,5419,4556,4517,3788
...,...,...,...,...,...,...,...,...,...,...,...,...
58,{'$oid': '607d4c2708e81eceb6b6e3bf'},G0801170,Colorado,8,80,Summit County,117,1170,8848,12881,23548,27994
59,{'$oid': '607d4c2708e81eceb6b6e3c4'},G0801190,Colorado,8,80,Teller County,119,1190,8034,12468,20555,23350
60,{'$oid': '607d4c2708e81eceb6b6e3c3'},G0801210,Colorado,8,80,Washington County,121,1210,5304,4812,4926,4814
61,{'$oid': '607d4c2708e81eceb6b6e3c2'},G0801230,Colorado,8,80,Weld County,123,1230,123438,131821,180936,252825


In [17]:
populationLabelsFrame = pd.io.json.read_json('../data/county_total_population.Colorado/fieldLabels.json')
populationLabelsFrame

""


In [18]:
populationGeometryFrame = pd.io.json.read_json('../data/county_total_population.Colorado/linkedGeometry.json')
populationGeometryFrame

,_id,type,geometry,properties,GISJOIN
0,{'$oid': '604964ecfed2e95d6b68b676'},Feature,"{'type': 'Polygon', 'coordinates': [[[-104.961...","{'STATEFP10': '08', 'COUNTYFP10': '001', 'COUN...",G0800010
1,{'$oid': '604964ecfed2e95d6b68b684'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.457...","{'STATEFP10': '08', 'COUNTYFP10': '003', 'COUN...",G0800030
2,{'$oid': '604964eefed2e95d6b68bb16'},Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...","{'STATEFP10': '08', 'COUNTYFP10': '005', 'COUN...",G0800050
3,{'$oid': '604964eefed2e95d6b68b9c8'},Feature,"{'type': 'Polygon', 'coordinates': [[[-107.128...","{'STATEFP10': '08', 'COUNTYFP10': '007', 'COUN...",G0800070
4,{'$oid': '604964eefed2e95d6b68b83d'},Feature,"{'type': 'Polygon', 'coordinates': [[[-102.747...","{'STATEFP10': '08', 'COUNTYFP10': '009', 'COUN...",G0800090
...,...,...,...,...,...
59,{'$oid': '604964ecfed2e95d6b68b677'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.924...","{'STATEFP10': '08', 'COUNTYFP10': '117', 'COUN...",G0801170
60,{'$oid': '604964eefed2e95d6b68b840'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.329...","{'STATEFP10': '08', 'COUNTYFP10': '119', 'COUN...",G0801190
61,{'$oid': '604964eefed2e95d6b68b9c5'},Feature,"{'type': 'Polygon', 'coordinates': [[[-102.779...","{'STATEFP10': '08', 'COUNTYFP10': '121', 'COUN...",G0801210
62,{'$oid': '604964eefed2e95d6b68b9c3'},Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...","{'STATEFP10': '08', 'COUNTYFP10': '123', 'COUN...",G0801230


## Data Exploration

In [ ]:
sns.distplot(covidDataFrame.newCaseCount)

In [ ]:
sns.distplot(temperatureDataFrame.tempSingleMean)

## Preprocessing

In [21]:
covidDataFrame.loc[covidDataFrame['_id'] == '{\'$oid\': \'615fc624bf06594c4cad5b34\'}']

,_id,GISJOIN,epoch_time,dateString,county,state,totalCaseCount,newCaseCount,totalDeathCount,newDeathCount


In [22]:
covidFlatDataFrame = pd.json_normalize(json.load(open())

AttributeError: 'str' object has no attribute 'values'

In [19]:
covidDataFrame.loc[covidDataFrame['GISJOIN'] == 'G0800970']

,_id,GISJOIN,epoch_time,dateString,county,state,totalCaseCount,newCaseCount,totalDeathCount,newDeathCount
30576,{'$oid': '615fc627bf06594c4cadbc47'},G0800970,{'$numberLong': '1579935600000'},1/25/20,Pitkin,Colorado,0,0,0,0
30577,{'$oid': '615fc627bf06594c4cadbc49'},G0800970,{'$numberLong': '1580108400000'},1/27/20,Pitkin,Colorado,0,0,0,0
30578,{'$oid': '615fc627bf06594c4cadbc4c'},G0800970,{'$numberLong': '1580367600000'},1/30/20,Pitkin,Colorado,0,0,0,0
30579,{'$oid': '615fc627bf06594c4cadbc6d'},G0800970,{'$numberLong': '1583218800000'},3/3/20,Pitkin,Colorado,0,0,0,0
30580,{'$oid': '615fc627bf06594c4cadbc7b'},G0800970,{'$numberLong': '1584252000000'},3/15/20,Pitkin,Colorado,11,0,0,0
...,...,...,...,...,...,...,...,...,...,...
31195,{'$oid': '615fc627bf06594c4cadbe8d'},G0800970,{'$numberLong': '1630216800000'},8/29/21,Pitkin,Colorado,2805,0,4,0
31196,{'$oid': '615fc627bf06594c4cadbe96'},G0800970,{'$numberLong': '1631167200000'},9/9/21,Pitkin,Colorado,2877,9,4,0
31197,{'$oid': '615fc627bf06594c4cadbe98'},G0800970,{'$numberLong': '1630908000000'},9/6/21,Pitkin,Colorado,2858,11,4,0
31198,{'$oid': '615fc627bf06594c4cadbe9a'},G0800970,{'$numberLong': '1631512800000'},9/13/21,Pitkin,Colorado,2907,19,4,0


Create Dataframes:

In [ ]:
expected_directories = {
    'covid_county': os.path.join(dataPath, 'covid_county.Colorado'),
    'neon_2d_wind': os.path.join(dataPath, 'neon_2d_wind.Colorado'),
    'neon_single_asp_air_temperature': os.path.join(dataPath, 'neon_single_asp_air_temperature.Colorado'),
    'neon_barometric_pressure': os.path.join(dataPath, 'neon_barometric_pressure.Colorado'),
    'svi_county_GISJOIN': os.path.join(dataPath, 'svi_county_GISJOIN.Colorado')
}
covid_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['covid_county'], 'data.json'),
    os.path.join(expected_directories['covid_county'], 'fieldLabels.json'),
    os.path.join(expected_directories['covid_county'], 'linkedGeometry.json'),
    join_on_key='GISJOIN')
wind_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['neon_2d_wind'], 'data.json'),
    os.path.join(expected_directories['neon_2d_wind'], 'fieldLabels.json'),
    os.path.join(expected_directories['neon_2d_wind'], 'linkedGeometry.json'),
    join_on_key='site')
air_temp_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['neon_single_asp_air_temperature'], 'data.json'),
    os.path.join(expected_directories['neon_single_asp_air_temperature'], 'fieldLabels.json'),
    os.path.join(expected_directories['neon_single_asp_air_temperature'], 'linkedGeometry.json'),
    join_on_key='site')
air_presssure_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['neon_barometric_pressure'], 'data.json'),
    os.path.join(expected_directories['neon_barometric_pressure'], 'fieldLabels.json'),
    os.path.join(expected_directories['neon_barometric_pressure'], 'linkedGeometry.json'),
    join_on_key='site')
county_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['svi_county_GISJOIN'], 'data.json'),
    os.path.join(expected_directories['svi_county_GISJOIN'], 'fieldLabels.json'),
    os.path.join(expected_directories['svi_county_GISJOIN'], 'linkedGeometry.json'),
    join_on_key='GISJOIN')

print(covid_df.info)
print(wind_df.info)
print(air_temp_df.info)
print(air_presssure_df.info)
print(county_df.info)



## References

## About this Notebook

**Authors:** Kyle Bassignani, Jeff Borgerson, and Christian Westbrook  
**Updated On:** 2021-11-12